In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sms = pd.read_csv('SMSSpamCollection', sep='\t',header=None)
sms.columns =['tag','sms']
sms.head(10)

,tag,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [3]:
# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
import spacy
import re

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

# Convert to list
data = sms.sms.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

C:\Users\anirban.c\AppData\Local\Continuum\anaconda3\envs\anirban\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'great', 'world', 'la', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']]


In [5]:
import warnings
warnings.filterwarnings('ignore')

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [6]:
from nltk.corpus import stopwords
from collections import Counter
stop_words = stopwords.words('english')
import spacy

def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

In [7]:
data_ready = str(process_words(data_words))  # processed Text Data!

In [8]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(data_ready)

doc = nlp(data_ready)

ner_list = []
ner_list_label = []
ner_list_explain = []

for ent in doc.ents:
            ner_list.append(ent.text)
            ner_list_label.append(ent.label_)
            ner_list_explain.append(spacy.explain(ent.label_))
            
            
ner_list = pd.DataFrame(ner_list)
ner_list_label = pd.DataFrame(ner_list_label)
ner_list_explain = pd.DataFrame(ner_list_explain)

ner_entities=pd.concat([ner_list,ner_list_label,ner_list_explain],axis=1)
ner_entities.columns=['ner_text','ner_label','label_explain']
ner_entities

,ner_text,ner_label,label_explain
0,tonight,TIME,Times smaller than a day
1,today,DATE,Absolute or relative dates or periods
2,first,ORDINAL,"""first"", ""second"", etc."
3,first,ORDINAL,"""first"", ""second"", etc."
4,eighth,ORDINAL,"""first"", ""second"", etc."
...,...,...,...
360,first,ORDINAL,"""first"", ""second"", etc."
361,today,DATE,Absolute or relative dates or periods
362,tonight,TIME,Times smaller than a day
363,tomorrow,DATE,Absolute or relative dates or periods


In [9]:
ner_entities.ner_label.value_counts()

DATE        179
TIME         95
ORDINAL      87
NORP          3
CARDINAL      1
Name: ner_label, dtype: int64

In [10]:
pos_list = []
pos_list_label = []
pos_list_tag = []
pos_list_explain = []
pos_list_dep = []

for token in doc:
            pos_list.append(token.text)
            pos_list_label.append(token.pos_)
            pos_list_tag.append(token.tag_)
            pos_list_explain.append(spacy.explain(token.tag_))
            pos_list_dep.append(token.dep_)
            


In [11]:
pos_list = pd.DataFrame(pos_list)
pos_list_label = pd.DataFrame(pos_list_label)
pos_list_tag = pd.DataFrame(pos_list_tag)
pos_list_explain = pd.DataFrame(pos_list_explain)
pos_list_dep = pd.DataFrame(pos_list_dep)

pos_entities=pd.concat([pos_list, pos_list_label, pos_list_tag, pos_list_explain, pos_list_dep],axis=1)
pos_entities.columns=['text','pos_label','pos_tag','pos_explain','dependency']

pos_entities = pos_entities[(pos_entities.pos_label != 'PUNCT')].reset_index(drop=True)
pos_entities

,text,pos_label,pos_tag,pos_explain,dependency
0,go,VERB,VB,"verb, base form",ccomp
1,jurong,NOUN,NN,"noun, singular or mass",dep
2,point,NOUN,NN,"noun, singular or mass",conj
3,crazy,ADJ,JJ,adjective,amod
4,available,ADJ,JJ,adjective,amod
...,...,...,...,...,...
40376,give,VERB,VB,"verb, base form",dep
40377,free,ADJ,JJ,adjective,conj
40378,rofl,NOUN,NNS,"noun, plural",npadvmod
40379,true,ADJ,JJ,adjective,amod


In [12]:
pos_entities.pos_label.value_counts()

NOUN     23012
VERB      8192
ADJ       4991
ADV       2195
INTJ      1131
X          406
PROPN      268
ADP        125
NUM         24
DET         19
PRON         8
PART         6
CCONJ        3
AUX          1
Name: pos_label, dtype: int64